In [6]:
%%time
#This funtion returns the list of "Returns" of dataframe "df" for each group and its "sum" for "d" days.
def getReturns(df,d,g):
    reward_list = df[(df["Day"]<=d)&(df["Group"]==g)].Returns.tolist()
    return (sum(reward_list),reward_list)

#------------------------------------------------------------------------------------------------------------#

# This function returns the list of "Actions" taken in each group of dataframe"df" and its "sum" for "d" days.
def getActions(df,d,g):
    action_list = df[(df["Day"]<=d)&(df["Group"]==g)].Actions.tolist()
    return (sum(action_list),action_list)

#------------------------------------------------------------------------------------------------------------#

# this function returns the Conversion Rate ie simply the returns/ actions 
def conversionRate(meth):
    return meth[1]/meth[0]

#------------------------------------------------------------------------------------------------------------#

#This function evaluates the standard error of the test method 
def standard_error(crb,n):
    return math.sqrt((crb * (1 - crb)) / n)

#------------------------------------------------------------------------------------------------------------#

#This funtion evaluates the percentage difference so that we can check the status of the test method
def per_diff(a,b):
    return ((b-a)/a)

#------------------------------------------------------------------------------------------------------------#

#this function is used for Z score so that we can check the p-value later on for statistical significance
def z_score(cra, crb, sea, seb):
    return (crb - cra) / math.sqrt(seb**2 + sea**2)
    
#------------------------------------------------------------------------------------------------------------#    

#P-value of the test group tells us about the statistical significance of the test method
def p_value(z_score):
    return scipy.stats.norm.sf(abs(z_score))

#------------------------------------------------------------------------------------------------------------#

#If we came to the conclusion of choosing any specific method, this function will edit the data frame as per out requirement

def implement_dec_B(df,i,g,a,b,param):
    #print("I am here")
    #print(df.loc[((df["Day"]>=i) & (df["Group"]==g) & (df["Method"]=="b")),param].shape)
    #print(df[(df["Day"]>=i)&(df["Group"]==g)&(df["Method"]==a)][param].shape)
    #df.loc[((df["Day"]==i) & (df["Group"]==g) & (df["Method"]==b)),param] = df[(df["Day"]==i)&(df["Group"]==g)&(df["Method"]==a)][param].tolist()
    df.loc[((df["Day"]>=i) & (df["Group"]==g) & (df["Method"]==b)),param] = df[(df["Day"]>=i)&(df["Group"]==g)&(df["Method"]==a)][param].tolist()
    
#------------------------------------------------------------------------------------------------------------#   


"""
How it works? : The first forloop gives the group number on which we apply the program we have made, in the seconf 
farloop we have the days so that for each day we can know how much we have got in returns for both A and B separately
so that later on we can find which one is best,
I have taken a minimum of 10 days to start with each group so for each method in one group i will start with 
10 days for Action and same for Returns and evaluate the conversion rate which i will store in an array meth A and meth B
and similarly we have an array to store the percentage difference and p-val until atleast" 150 actions" have been taken 
by each method or it is less than that for "45 days". and if any group qualifies any of them, then we choose either A or B 
based on the conditon. There was one more case wherein both A and B were 0 for 45 days so in that case we can choose any of
the method. I have taken a threshold of 20% in percentage difference and 0.05 for p-value.

At the end of the program we are generating a file which tells us which method we have chosen for which group and
what are the returns before and after.
"""


import math
import scipy.stats
import statistics
import pandas as pd

df = pd.read_csv("data.csv")
days = df.Day.unique().tolist()
groups = df.Group.unique().tolist()
df_a = df[df["Method"] == "A"]
df_b = df[df["Method"] == "B"]

before = []
after = []
groupx = []
final_method = []

for g in groups:
    print("Group",g)
    pd_avg = []
    pval_avg = []
    flag = 0
    
    groupx.append(g)
    before.append(df[(df["Group"]==g)]["Returns"].sum())
    print("Before",df[(df["Group"]==g)]["Returns"].sum())
    
    for i in days[10:]:
        if (flag == 1) or (flag == 3):
            implement_dec_B(df,i,g,"A","B","Returns")
            implement_dec_B(df,i,g,"A","B","Method")
            final_method.append("A")
            break

        if flag == 2:
            implement_dec_B(df,i,g,"B","A","Returns")
            implement_dec_B(df,i,g,"B","A","Method")
            final_method.append("B")
            break
        #remove this code later

        meth_a = []
        meth_b = []

        aca , action_list_a = getActions(df_a,i,g)
        rwa , reward_list_a = getReturns(df_a,i,g)
        acb , action_list_b = getActions(df_b,i,g)
        rwb , reward_list_b = getReturns(df_b,i,g)

        meth_a.append(aca)
        meth_a.append(rwa)
        meth_b.append(acb)
        meth_b.append(rwb)

        cra = conversionRate(meth_a)
        crb = conversionRate(meth_b)

        #to make sure that cra is not zero
        if cra ==0:
            if i <45:
                print("Choose Any Method")
                flag = 1
            continue
            
        percentage_difference = per_diff(cra,crb)
        pd_avg.append(percentage_difference)

        seb = standard_error(crb,meth_b[0])
        sea = standard_error(cra,meth_a[0])

        z = z_score(cra,crb,sea,seb)
        pval = p_value(z)
        pval_avg.append(pval)
        
        #set threshold, min 123 but takes too long 
        if (aca >150) or (i > 45):
            param1 = statistics.mean(pd_avg)
            param2 = statistics.mean(pval_avg)
            
            #set your threshold
            if ((param1 > .1) and (param2< .05)):
                print("Choose method B")
                flag = 2
            else:
                print("Choose method A")
                flag = 3
    
    after.append(df[(df["Group"]==g)]["Returns"].sum())
    print("After",df[(df["Group"]==g)]["Returns"].sum())
    print()

#------------------------------------------------------------------------------------------------------------#
    
new = pd.DataFrame()
new["Group"] = groupx
new["Method"] = final_method
new["Before"] = before
new["After"] = after


new["Success"] = ""
new.loc[new["Before"] <= new["After"], "Success"] = 1
new.loc[new["Before"] > new["After"], "Success"] = 0
effi = new["Success"].value_counts()

new.to_excel("output.xlsx")


print("The Returns before using the decision making algorithm: ",new["Before"].sum()) 
print("The Returns after using the decsion making algorithm: ", new["After"].sum())
print()
print("Improvements in performance after applying the decision making algorithm:", str(((new["After"].sum() - new["Before"].sum()) / new["Before"].sum())*100)[:4],"%")
print("The efficiency of Algorithm is around: ",round(effi[1]/effi.sum() *100),"%")

Group 0
Before 337
Choose method A
After 539

Group 1
Before 492
Choose method A
After 855

Group 2
Before 47
Choose method A
After 77

Group 3
Before 152
Choose method A
After 132

Group 4
Before 910
Choose method B
After 1169

Group 5
Before 866
Choose method A
After 1111

Group 6
Before 1125
Choose method B
After 1624

Group 7
Before 1043
Choose method A
After 911

Group 8
Before 82
Choose method A
After 105

Group 9
Before 1542
Choose method B
After 1594

Group 10
Before 1687
Choose method A
After 1666

Group 11
Before 214
Choose method A
After 177

Group 12
Before 933
Choose method A
After 967

Group 13
Before 544
Choose method A
After 538

Group 14
Before 1177
Choose method B
After 1603

Group 15
Before 427
Choose method A
After 732

Group 16
Before 354
Choose method A
After 482

Group 17
Before 642
Choose method A
After 816

Group 18
Before 310
Choose method A
After 458

Group 19
Before 890
Choose method A
After 879

Group 20
Before 676
Choose method B
After 859

Group 21
Before

After 257

Group 173
Before 337
Choose method B
After 431

Group 174
Before 434
Choose method A
After 447

Group 175
Before 1011
Choose method A
After 1278

Group 176
Before 384
Choose method A
After 605

Group 177
Before 11
Choose Any Method
After 22

Group 178
Before 1269
Choose method B
After 1842

Group 179
Before 143
Choose method A
After 137

Group 180
Before 395
Choose method B
After 586

Group 181
Before 817
Choose method A
After 705

Group 182
Before 482
Choose method A
After 615

Group 183
Before 705
Choose method A
After 602

Group 184
Before 397
Choose method A
After 297

Group 185
Before 742
Choose method A
After 947

Group 186
Before 101
Choose method A
After 107

Group 187
Before 680
Choose method B
After 1115

Group 188
Before 44
Choose method A
After 43

Group 189
Before 679
Choose method B
After 927

Group 190
Before 910
Choose method A
After 910

Group 191
Before 70
Choose method B
After 99

Group 192
Before 302
Choose method B
After 406

Group 193
Before 320
Choose 

After 1172

Group 345
Before 861
Choose method B
After 1156

Group 346
Before 839
Choose method B
After 1061

Group 347
Before 232
Choose method A
After 377

Group 348
Before 277
Choose method A
After 352

Group 349
Before 576
Choose method B
After 826

Group 350
Before 772
Choose method A
After 1090

Group 351
Before 161
Choose method B
After 168

Group 352
Before 867
Choose method A
After 716

Group 353
Before 24
Choose method A
After 29

Group 354
Before 117
Choose method A
After 124

Group 355
Before 130
Choose Any Method
After 16

Group 356
Before 497
Choose method A
After 816

Group 357
Before 1015
Choose method A
After 1262

Group 358
Before 189
Choose method A
After 289

Group 359
Before 119
Choose method B
After 153

Group 360
Before 231
Choose method B
After 389

Group 361
Before 1213
Choose method A
After 1095

Group 362
Before 663
Choose method A
After 812

Group 363
Before 1225
Choose method A
After 1257

Group 364
Before 280
Choose method A
After 400

Group 365
Before 86


After 418

Group 517
Before 720
Choose method A
After 627

Group 518
Before 293
Choose method B
After 398

Group 519
Before 391
Choose method A
After 488

Group 520
Before 655
Choose method A
After 937

Group 521
Before 375
Choose method A
After 418

Group 522
Before 223
Choose method A
After 215

Group 523
Before 625
Choose method B
After 929

Group 524
Before 119
Choose method B
After 162

Group 525
Before 835
Choose method B
After 1405

Group 526
Before 369
Choose method B
After 453

Group 527
Before 25
Choose method A
After 26

Group 528
Before 583
Choose method A
After 824

Group 529
Before 16
Choose method A
After 24

Group 530
Before 125
Choose method B
After 170

Group 531
Before 765
Choose method A
After 711

Group 532
Before 688
Choose method B
After 905

Group 533
Before 86
Choose method A
After 117

Group 534
Before 941
Choose method B
After 1273

Group 535
Before 20
Choose Any Method
After 3

Group 536
Before 501
Choose method B
After 858

Group 537
Before 686
Choose metho

Choose method B
After 1218

Group 690
Before 176
Choose method A
After 261

Group 691
Before 72
Choose method A
After 101

Group 692
Before 163
Choose method A
After 210

Group 693
Before 783
Choose method A
After 744

Group 694
Before 758
Choose method A
After 739

Group 695
Before 88
Choose Any Method
After 7

Group 696
Before 136
Choose method B
After 174

Group 697
Before 1554
Choose method A
After 1515

Group 698
Before 1351
Choose method A
After 1578

Group 699
Before 270
Choose method A
After 196

Group 700
Before 268
Choose method B
After 289

Group 701
Before 651
Choose method A
After 620

Group 702
Before 623
Choose method A
After 995

Group 703
Before 518
Choose method A
After 844

Group 704
Before 689
Choose method A
After 937

Group 705
Before 0
Choose Any Method
After 0

Group 706
Before 91
Choose method A
After 124

Group 707
Before 582
Choose method A
After 606

Group 708
Before 1159
Choose method B
After 1593

Group 709
Before 1030
Choose method A
After 1046

Group 710

After 246

Group 860
Before 773
Choose method A
After 1017

Group 861
Before 584
Choose method B
After 808

Group 862
Before 910
Choose method A
After 1252

Group 863
Before 701
Choose method A
After 1151

Group 864
Before 98
Choose method B
After 117

Group 865
Before 955
Choose method A
After 1370

Group 866
Before 518
Choose method B
After 859

Group 867
Before 5
Choose Any Method
After 1

Group 868
Before 204
Choose method B
After 252

Group 869
Before 948
Choose method B
After 1156

Group 870
Before 615
Choose method B
After 841

Group 871
Before 1153
Choose method A
After 1656

Group 872
Before 255
Choose method A
After 302

Group 873
Before 778
Choose method B
After 1164

Group 874
Before 1166
Choose method A
After 1233

Group 875
Before 388
Choose method A
After 337

Group 876
Before 121
Choose method A
After 154

Group 877
Before 727
Choose method A
After 924

Group 878
Before 25
Choose method A
After 37

Group 879
Before 583
Choose method B
After 1025

Group 880
Before 837
Ch